In [ ]:
import tweepy
import csv
import sys
import jsonpickle
import os
import time

#joanna's twitter keys, used to get data


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

####
# adapted from http://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively./
####

#this is our query
searchQuery = '#demdebate OR debate hillary OR clinton OR hilary OR bernie OR bern OR sanders OR martin OR omalley'  # this is what we're searching for
maxTweets = 1000000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets.txt' # We'll store the tweets in a text file.


# this is our early end donald trump tweet
sinceId = 665673643637215233

# this is out late end donald trump tweet
max_id = 665775632077451264

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                        '\n')
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        #handle the rate limit
        #to get more than 15k tweets
        except tweepy.TweepError as e:
            print 'RATE LIMIT - waiting 15 minutes...'
            #sleep to waite out the rate limit
            time.sleep(60*15)
            continue
            
            
print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))
